<a href="https://colab.research.google.com/github/Thayyebsalim/music_analyisis/blob/master/Music_Sentimental_analysis_NLP_by_Tone_and_Lyrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro

The idea of this code is to find the mood of the song in any way possible helping us to segregate songs better.

1.   First part is gonna be scraping the lyrics from any website or API 
2.   Finding the vibe of the song with sentimental analysis
3.   Fetch and mp3 version and check the mood with transfer learning hopefully or something similar



In [1]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Hit:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Fetched 252 kB in 2s (142 kB/s)
Reading package lists... Done
Reading package lists... Done
Building de

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import re
import time
import string
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download("vader_lexicon")
nltk.download("brown")
nltk.download('stopwords')
nltk.download("wordnet")
nltk.download("punkt")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

We need to scrape the song lyrics from websites or API to use here

I am using selenium here and the speed is not the best at the moment so I might switch to scrapy or use an API

In [3]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


def get_track_link(track_name): # Gets track name from user and converts it so that we can use it in a link
  #track_name = input("Enter a track name: ")
  track_name_link = track_name.rstrip().replace(" ","%20").lower()
  return track_name_link


def scrape_tracklist(track_link): # Scrapes the page for list of tracks with similar names or such 
  driver = webdriver.Chrome(chrome_options=chrome_options)
  driver.get("https://www.musixmatch.com/search/" + track_link + "/tracks")
  element = driver.find_element_by_class_name("box-content")
  link_list = [i.get_attribute('href') for i in element.find_elements_by_tag_name('a')]
  driver.close()
  lyric_links=[i for i in link_list if re.search("/lyrics/",i)] #could do it in one line but for the sake of clarity, I'm gonna do two
  lyric_links=[i for i in lyric_links if not re.search("/add",i)]
  return lyric_links


def scrape_lyrics(lyric_links_list,choice): # Scrapes the lyrics element
  driver = webdriver.Chrome(chrome_options=chrome_options)
  driver.get(lyric_links_list[choice])
  lyric_elem=driver.find_elements_by_class_name('mxm-lyrics__content')
  lyric_elem= [i.find_element_by_tag_name('span') for i in lyric_elem] 
  lyric_elem= [i.get_attribute('innerHTML') for i in lyric_elem]
  driver.close()
  return lyric_elem


def select_track(artist_track_list): # Lets the user select a track
  sel=0
  for track in artist_track_list:
    print(f"{track[0]+1} {track[1]} - {track[2]}")
  sel=int(input('Enter the number of your desired track: '))-1
  return sel


def get_lyric_element(track_link,choice=0): # Cleans up the lyric element and returns a cleaned list(array) with just lyrics inside
  lyric_links_list = scrape_tracklist(track_link)
  print(lyric_links_list)
  artist_track_list = [[lyric_links_list.index(x), x.split("/")[4],x.split('/')[5]] for x in lyric_links_list]
  print(artist_track_list)
  if len(artist_track_list) > 1 and choice != "default":
    choice = select_track(artist_track_list)
  elif len(artist_track_list) == 0 :
    return "Sorry! no such song"
  else:
    choice=0
  lyric_element = scrape_lyrics(lyric_links_list,choice)
  return " ".join(lyric_element)



In [4]:
track_name= "grandson war"
track_link= get_track_link(track_name) #uncomment to activate function
track_link

'grandson%20war'

In [5]:
lyric_element = get_lyric_element(track_link,"default")
lyric_element

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


['https://www.musixmatch.com/lyrics/Grandson/War']
[[0, 'Grandson', 'War']]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: use options instead of chrome_options


"Until the philosophy, we chose one race superior\nAnd another\nInferior He's finally\nAnd permanently\nDiscredited and abandoned\n\nAnd it's war in the east\nWar in the west\nWar up north\nWar down south\n\nAnd it's war in the east\nWar in the west\nWar up north\nWar down south\n\nEverywhere is war\nIt's a war\nOh, everywhere is war\nIt's a war\n\nAnd until there's no longer\nFirst and second class citizens of any nation\nIt's on\nAnd until the color of a man's skin serves no more significance\nThan the color of his eyes\nWe say war\n\nAnd it's war in the east\nWar in the west\nWar up north\nWar down south\n\nAnd it's war in the east\nWar in the west\nWar up north\nWar down south\nEverywhere is\nOh, everywhere is war\n\nIt's a war\nOh, everywhere is war\nIt's a war\nOh, everywhere is war\n\nLa, la, la, la, la, la, la\nLa, la, la, la, la, la, la\n(Everywhere is, everywhere is, everywhere is)\n(Everywhere is, everywhere is, everywhere is)\n\nEverywhere is war\nEverywhere is war"

The first part is done, web scraper should work fine. I did not implement error cases for now becuase I want to focus on the next two parts of the project


In [6]:
def filter_raw_lyrics(raw_lyr): #The scraped element has blanks and \n's lets filter those 
  return [i for i in raw_lyr.split("\n") if i != '']

#joiner
def convert_arr_to_str(arr):
  return " ".join(arr)

In [7]:
lyrics_as_array= filter_raw_lyrics(lyric_element)
lyrics=convert_arr_to_str(lyrics_as_array) #returns a single string as lyrics
lyrics,lyrics_as_array

("Until the philosophy, we chose one race superior And another Inferior He's finally And permanently Discredited and abandoned And it's war in the east War in the west War up north War down south And it's war in the east War in the west War up north War down south Everywhere is war It's a war Oh, everywhere is war It's a war And until there's no longer First and second class citizens of any nation It's on And until the color of a man's skin serves no more significance Than the color of his eyes We say war And it's war in the east War in the west War up north War down south And it's war in the east War in the west War up north War down south Everywhere is Oh, everywhere is war It's a war Oh, everywhere is war It's a war Oh, everywhere is war La, la, la, la, la, la, la La, la, la, la, la, la, la (Everywhere is, everywhere is, everywhere is) (Everywhere is, everywhere is, everywhere is) Everywhere is war Everywhere is war",
 ['Until the philosophy, we chose one race superior',
  'And anot

Well lets have some fun with the lyrics

In [8]:
def swfilter(word): #function we use in filter()
  return word not in stopwords.words()

def remove_stopword(lyr_arr):
  return [" ".join(filter(swfilter,sentence.split())) for sentence in lyr_arr]

In [9]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

def map_contraction(word): #function we use in map()
  word=word.lower()
  if word in contractions:
    return contractions[word]
  else: 
    return word

def remove_contraction(lyr_arr):
  return [" ".join(map(map_contraction, sentence.split())) for sentence in lyr_arr]

In [10]:
wordnet_lemmatizer = WordNetLemmatizer()

def map_lemmatize(word):
  return wordnet_lemmatizer.lemmatize(word, pos="v")

def lemmatize_words(lyr_arr):
  return [" ".join(map(map_lemmatize, sentence.split())) for sentence in lyr_arr]

In [ ]:
lyrics_as_array_without_sw = remove_stopword(lyrics_as_array)
lyrics_without_sw = convert_arr_to_str(lyrics_as_array_without_sw)
lyrics_as_array_without_sw,lyrics_without_sw

(['Until philosophy, chose race superior',
  'And another',
  "Inferior He's finally",
  'And permanently',
  'Discredited abandoned',
  'And east',
  'War west',
  'War north',
  'War south',
  'And east',
  'War west',
  'War north',
  'War south',
  'Everywhere',
  "It's",
  'Oh, everywhere',
  "It's",
  "And there's longer",
  'First second class citizens nation',
  "It's",
  "And color man's skin serves significance",
  'Than color eyes',
  'We say',
  'And east',
  'War west',
  'War north',
  'War south',
  'And east',
  'War west',
  'War north',
  'War south',
  'Everywhere',
  'Oh, everywhere',
  "It's",
  'Oh, everywhere',
  "It's",
  'Oh, everywhere',
  'La, la, la, la, la, la,',
  'La, la, la, la, la, la,',
  '(Everywhere is, everywhere is, everywhere is)',
  '(Everywhere is, everywhere is, everywhere is)',
  'Everywhere',
  'Everywhere'],
 "Until philosophy, chose race superior And another Inferior He's finally And permanently Discredited abandoned And east War west War n

In [ ]:
lyrics_as_array_without_contraction = remove_contraction(lyrics_as_array)
lyrics_without_contraction = convert_arr_to_str(lyrics_as_array_without_contraction)
lyrics_as_array_without_contraction,lyrics_without_contraction

(['until the philosophy, we chose one race superior',
  'and another',
  'inferior he has / he is finally',
  'and permanently',
  'discredited and abandoned',
  'and it has / it is war in the east',
  'war in the west',
  'war up north',
  'war down south',
  'and it has / it is war in the east',
  'war in the west',
  'war up north',
  'war down south',
  'everywhere is war',
  'it has / it is a war',
  'oh, everywhere is war',
  'it has / it is a war',
  'and until there has / there is no longer',
  'first and second class citizens of any nation',
  'it has / it is on',
  "and until the color of a man's skin serves no more significance",
  'than the color of his eyes',
  'we say war',
  'and it has / it is war in the east',
  'war in the west',
  'war up north',
  'war down south',
  'and it has / it is war in the east',
  'war in the west',
  'war up north',
  'war down south',
  'everywhere is',
  'oh, everywhere is war',
  'it has / it is a war',
  'oh, everywhere is war',
  'it 

In [ ]:
lyrics_as_array_with_lemmatized_words=lemmatize_words(lyrics_as_array)
lyrics_with_lemmatized_words=convert_arr_to_str(lyrics_as_array_with_lemmatized_words)
lyrics_as_array_with_lemmatized_words,lyrics_with_lemmatized_words

(['Until the philosophy, we choose one race superior',
  'And another',
  "Inferior He's finally",
  'And permanently',
  'Discredited and abandon',
  "And it's war in the east",
  'War in the west',
  'War up north',
  'War down south',
  "And it's war in the east",
  'War in the west',
  'War up north',
  'War down south',
  'Everywhere be war',
  "It's a war",
  'Oh, everywhere be war',
  "It's a war",
  "And until there's no longer",
  'First and second class citizens of any nation',
  "It's on",
  "And until the color of a man's skin serve no more significance",
  'Than the color of his eye',
  'We say war',
  "And it's war in the east",
  'War in the west',
  'War up north',
  'War down south',
  "And it's war in the east",
  'War in the west',
  'War up north',
  'War down south',
  'Everywhere be',
  'Oh, everywhere be war',
  "It's a war",
  'Oh, everywhere be war',
  "It's a war",
  'Oh, everywhere be war',
  'La, la, la, la, la, la, la',
  'La, la, la, la, la, la, la',
  '(E


Next we have to tackle the sentimantal analysis part. We aim to try bag of words then use NLTK on every line might be slow but I like to give prominence to accuracy

In [11]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia=SentimentIntensityAnalyzer()
def get_vader_score(lyr):
  # Polarity score returns dictionary
  sentiment = sia.polarity_scores(lyr)
  return sentiment['compound']
  """
  print(ss)
  print(ss['compound'])
  ss.pop("compound")
  print("ss=",ss)
  x=sorted(ss.values(), reverse=True)[0]
  for i in ss.keys():
    if ss[i]==x:
      if "i" == "neg":
        ss[i]=-(ss[i])
      print(ss[i])
  """  

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
get_vader_score(lyrics)

-0.999

In [12]:
def get_textblob_score(lyr):
  blob=TextBlob(lyr)
  return blob.polarity

In [ ]:
get_textblob_score(lyrics)

0.008641975308641962

In [71]:
!pip install watson_developer_cloud
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions, SentimentOptions, CategoriesOptions
natural_language_understanding = NaturalLanguageUnderstandingV1(                                         
    version='2018-11-16',
    iam_apikey="hzkkbjivKNHfoi3cFlMBOPG6-YHLc-7CAQKwXn8Rt7JT", 
    url="https://api.eu-gb.natural-language-understanding.watson.cloud.ibm.com/instances/512aa79c-a8ba-43fa-b9d5-a4382099b06f" # paste the url here
    )
def get_watson_score(input_text): 
  try:
    response = natural_language_understanding.analyze (
    text = input_text,
    features = Features(sentiment=SentimentOptions())).get_result()
    res = response.get('sentiment').get('document').get('score')
    return res
  except Exception as exception:
    print("error",input_text,type(exception).__name__)
    return 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  import sys


In [14]:
get_watson_score(lyrics)

-0.777161

In [ ]:
sentiment_score_fns=[get_textblob_score,get_vader_score]

In [20]:
# Returns all transformed arrays and further maps the arrays 

def get_lyric_arrays(lyr_arr):
  return [lyr_arr,
          remove_stopword(lyr_arr),
          remove_contraction(lyr_arr),
          lemmatize_words(lyr_arr),
          remove_contraction(remove_stopword(lyr_arr)),
          lemmatize_words(remove_stopword(lyr_arr)),
          remove_contraction(lemmatize_words(lyr_arr)),
          lemmatize_words(remove_contraction(remove_stopword(lyr_arr)))]

def get_lyrics(arr):
  return list(map(convert_arr_to_str,arr))


In [21]:

list_of_converted_lyr_arr=get_lyric_arrays(lyrics_as_array) #returns a tuple with 3 items lyrics, lyrics without stop wordsm 
list_of_converted_lyr=get_lyrics(list_of_converted_lyr_arr)

In [22]:
import pandas as pd 
import numpy as np

In [24]:
df=pd.DataFrame(index=["Lyrics","SW","CO","LM","SW-CO","SW-LM","CO-LM","SW-CO-LM"],columns=["Textblob","Textblob per line","Textblob nonzero","Vader","Vader per line","Vader nonzero","Watson","Watson per line","Watson nonzero"])
df

,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero
Lyrics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SW-CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SW-LM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CO-LM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SW-CO-LM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df["Textblob"]=list(map(get_textblob_score,list_of_converted_lyr))
df["Vader"]=list(map(get_vader_score,list_of_converted_lyr))
df["Watson"]=list(map(get_watson_score,list_of_converted_lyr))

In [26]:
textblob_per_line=[]
for trans in list_of_converted_lyr_arr:
  textblob_per_line.append(list(map(get_textblob_score,trans)))

df["Textblob per line"]=list(map(np.mean,textblob_per_line))
#textblob_per_line

In [27]:
vader_per_line=[]
for trans in list_of_converted_lyr_arr:
  vader_per_line.append(list(map(get_vader_score,trans)))

df["Vader per line"]=list(map(np.mean,vader_per_line))
#vader_per_line

In [35]:
def watson_min_words_arr(arr): #watson has minimum requirement to perform sentiment analysis
  x="" 
  watson_arr=[]
  for line in arr:
    if len(line.split())+len(x.split())<4:
      x=x+line+" "
      continue
    if x!= "" : line=x+line
    x=""
    watson_arr.append(line)
  return watson_arr 



In [40]:
converted_watson_arr=list(map(watson_min_words_arr,list_of_converted_lyr_arr))
converted_watson_arr

[['Until the philosophy, we chose one race superior',
  "And another Inferior He's finally",
  'And permanently Discredited and abandoned',
  "And it's war in the east",
  'War in the west',
  'War up north War down south',
  "And it's war in the east",
  'War in the west',
  'War up north War down south',
  "Everywhere is war It's a war",
  'Oh, everywhere is war',
  "It's a war And until there's no longer",
  'First and second class citizens of any nation',
  "It's on And until the color of a man's skin serves no more significance",
  'Than the color of his eyes',
  "We say war And it's war in the east",
  'War in the west',
  'War up north War down south',
  "And it's war in the east",
  'War in the west',
  'War up north War down south',
  'Everywhere is Oh, everywhere is war',
  "It's a war Oh, everywhere is war",
  "It's a war Oh, everywhere is war",
  'La, la, la, la, la, la, la',
  'La, la, la, la, la, la, la',
  '(Everywhere is, everywhere is, everywhere is)',
  '(Everywhere i

In [41]:
converted_watson_arr=list(map(watson_min_words_arr,list_of_converted_lyr_arr))

watson_per_line=[]
for trans in converted_watson_arr:
  watson_per_line.append(list(map(get_watson_score,trans)))

df["Watson per line"]=list(map(np.mean,watson_per_line))
#watson_per_line

error La, la, la, la, la, la, la
error La, la, la, la, la, la, la
error La, la, la, la, la, la,
error La, la, la, la, la, la,
error la, la, la, la, la, la, la
error la, la, la, la, la, la, la
error La, la, la, la, la, la, la
error La, la, la, la, la, la, la
error la, la, la, la, la, la,
error La, la, la, la, la, la,
error La, la, la, la, la, la,
error la, la, la, la, la, la, la
error la, la, la, la, la, la, la
error la, la, la, la, la, la,


In [43]:
print(len(watson_per_line),len(converted_watson_arr))
for i in range(8):
  print(len(watson_per_line[i]),len(converted_watson_arr[i]))

8 8
29 29
22 22
33 33
29 29
24 24
22 22
33 33
24 24


In [50]:
def mean_without_zero(arr):
  arr = [x for x in arr if x != 0]
  return np.mean(arr)

In [ ]:
df["Textblob nonzero"]=list(map(mean_without_zero,textblob_per_line))
df["Vader nonzero"]=list(map(mean_without_zero,vader_per_line))
df["Watson nonzero"]=list(map(mean_without_zero,watson_per_line))

In [ ]:
df

,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero
Lyrics,0.149415,0.046778,0.233889,0.9599,0.095550,0.281029,-0.778259,-0.118538,-0.269406
SW,0.196875,0.053000,0.331250,0.9894,0.100832,0.315100,-0.386954,-0.020349,-0.063953
CO,0.149415,0.046778,0.233889,0.9953,0.098372,0.289329,-0.773304,-0.111188,-0.252699
LM,0.147222,0.054778,0.248990,0.9659,0.099942,0.293947,-0.649890,-0.030253,-0.072030
SW-CO,0.196875,0.053000,0.331250,0.9953,0.103654,0.323919,-0.492278,-0.018453,-0.059314
SW-LM,0.186842,0.061000,0.338889,0.9905,0.107898,0.337181,-0.289106,0.010841,0.029814
CO-LM,0.147222,0.054778,0.248990,0.9960,0.102764,0.302247,-0.622290,-0.025782,-0.061385
SW-CO-LM,0.186842,0.061000,0.338889,0.9957,0.110720,0.346000,-0.356586,0.010238,0.028794


In [51]:
def get_df(lyr_arr):
  trans_lyr_arr=get_lyric_arrays(lyr_arr)
  trans_lyr=get_lyrics(trans_lyr_arr)
  df=pd.DataFrame(index=["Lyrics","SW","CO","LM","SW-CO","SW-LM","CO-LM","SW-CO-LM"],columns=["Textblob","Textblob per line","Textblob nonzero","Vader","Vader per line","Vader nonzero","Watson","Watson per line","Watson nonzero"])
  df["Textblob"]=list(map(get_textblob_score,trans_lyr))
  df["Vader"]=list(map(get_vader_score,trans_lyr))
  df["Watson"]=list(map(get_watson_score,trans_lyr))

  textblob_per_line=[]
  for trans in trans_lyr_arr:
    textblob_per_line.append(list(map(get_textblob_score,trans)))

  df["Textblob per line"]=list(map(np.mean,textblob_per_line))
  #textblob_per_line

  vader_per_line=[]
  for trans in trans_lyr_arr:
    vader_per_line.append(list(map(get_vader_score,trans)))

  df["Vader per line"]=list(map(np.mean,vader_per_line))
  #vader_per_line

  converted_watson_arr=list(map(watson_min_words_arr,trans_lyr_arr))

  watson_per_line=[]
  for trans in converted_watson_arr:
    print(trans)
    watson_per_line.append(list(map(get_watson_score,trans)))
    
  df["Watson per line"]=list(map(np.mean,watson_per_line))
  #watson_per_line

  df["Textblob nonzero"]=list(map(mean_without_zero,textblob_per_line))
  df["Vader nonzero"]=list(map(mean_without_zero,vader_per_line))
  df["Watson nonzero"]=list(map(mean_without_zero,watson_per_line))
  return df



In [59]:
songs=["blinding lights weeknd","jacob lee demons","grandson war","anson seabra emerald eyes","june messed up"]
song_arr=list(map(get_track_link,songs))
song_arr

['blinding%20lights%20weeknd',
 'jacob%20lee%20demons',
 'grandson%20war',
 'anson%20seabra%20emerald%20eyes',
 'june%20messed%20up']

In [60]:
elt_arr=[]
for i in song_arr:
  print(i)
  elt_arr.append(get_lyric_element(i,"default"))
elt_arr

blinding%20lights%20weeknd


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


['https://www.musixmatch.com/lyrics/The-Weeknd-3/Blinding-Lights', 'https://www.musixmatch.com/lyrics/Yssn-Nino/Blinding-Lights-Weeknd']
[[0, 'The-Weeknd-3', 'Blinding-Lights'], [1, 'Yssn-Nino', 'Blinding-Lights-Weeknd']]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: use options instead of chrome_options


jacob%20lee%20demons


...

june%20messed%20up


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


['https://www.musixmatch.com/lyrics/june-80/A-Little-Messed-Up', 'https://www.musixmatch.com/lyrics/R-J-Mischo-Charles-Wheal-Gary-Smith-Jr-Watson-June-Care-Mark-Hummel-Steve-Wolf-Kim-Wilson/You-Messed-Up']
[[0, 'june-80', 'A-Little-Messed-Up'], [1, 'R-J-Mischo-Charles-Wheal-Gary-Smith-Jr-Watson-June-Care-Mark-Hummel-Steve-Wolf-Kim-Wilson', 'You-Messed-Up']]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: use options instead of chrome_options


["Yeah\n\nI've been tryna call I've been on my own for long enough\nMaybe you can show me how to love, maybe\n\nI'm going through withdrawals\nYou don't even have to do too much\nYou can turn me on with just a touch, baby\n\nI look around and\nSin City's cold and empty (Oh)\nNo one's around to judge me (Oh)\nI can't see clearly when you're gone\n\nAnd I said, ooh, I'm blinded by the lights\nNo, I can't sleep until I feel your touch\nI said, ooh, I'm drowning in the night\nOh, when I'm like this, you're the one I trust\n(Hey, hey, hey)\n\nI'm running out of time\n'Cause I can see the sun light up the sky\nSo I hit the road in overdrive, baby, oh\n\nThe city's cold and empty (Oh)\nNo one's around to judge me (Oh)\nI can't see clearly when you're gone\n\nAnd I said, ooh, I'm blinded by the lights\nNo, I can't sleep until I feel your touch\nI said, ooh, I'm drowning in the night\nOh, when I'm like this, you're the one I trust\n\nI'm just calling back to let you know (Back to let you know)\

In [61]:
getdffrom=list(map(filter_raw_lyrics,elt_arr))
getdffrom

[['Yeah',
  "I've been tryna call I've been on my own for long enough",
  'Maybe you can show me how to love, maybe',
  "I'm going through withdrawals",
  "You don't even have to do too much",
  'You can turn me on with just a touch, baby',
  'I look around and',
  "Sin City's cold and empty (Oh)",
  "No one's around to judge me (Oh)",
  "I can't see clearly when you're gone",
  "And I said, ooh, I'm blinded by the lights",
  "No, I can't sleep until I feel your touch",
  "I said, ooh, I'm drowning in the night",
  "Oh, when I'm like this, you're the one I trust",
  '(Hey, hey, hey)',
  "I'm running out of time",
  "'Cause I can see the sun light up the sky",
  'So I hit the road in overdrive, baby, oh',
  "The city's cold and empty (Oh)",
  "No one's around to judge me (Oh)",
  "I can't see clearly when you're gone",
  "And I said, ooh, I'm blinded by the lights",
  "No, I can't sleep until I feel your touch",
  "I said, ooh, I'm drowning in the night",
  "Oh, when I'm like this, you'

In [62]:
x=list(map(get_df,getdffrom))

["Yeah I've been tryna call I've been on my own for long enough", 'Maybe you can show me how to love, maybe', "I'm going through withdrawals", "You don't even have to do too much", 'You can turn me on with just a touch, baby', 'I look around and', "Sin City's cold and empty (Oh)", "No one's around to judge me (Oh)", "I can't see clearly when you're gone", "And I said, ooh, I'm blinded by the lights", "No, I can't sleep until I feel your touch", "I said, ooh, I'm drowning in the night", "Oh, when I'm like this, you're the one I trust", "(Hey, hey, hey) I'm running out of time", "'Cause I can see the sun light up the sky", 'So I hit the road in overdrive, baby, oh', "The city's cold and empty (Oh)", "No one's around to judge me (Oh)", "I can't see clearly when you're gone", "And I said, ooh, I'm blinded by the lights", "No, I can't sleep until I feel your touch", "I said, ooh, I'm drowning in the night", "Oh, when I'm like this, you're the one I trust", "I'm just calling back to let you 

In [65]:
x[0]

,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero
Lyrics,0.032143,0.023039,0.097917,-0.4434,-0.027126,-0.065879,-0.882600,-0.271139,-0.336213
SW,-0.011538,0.016912,0.071875,-0.1940,-0.024274,-0.058950,-0.976902,-0.281368,-0.326387
CO,0.032143,0.023039,0.097917,-0.4434,-0.027126,-0.065879,-0.909600,-0.267845,-0.332128
LM,0.002273,0.017157,0.048611,-0.9644,-0.108050,-0.183685,-0.845551,-0.226736,-0.281153
SW-CO,-0.011538,0.016912,0.071875,-0.1940,-0.024274,-0.058950,-0.972905,-0.288342,-0.348414
SW-LM,-0.026190,0.011029,0.031250,-0.9582,-0.105197,-0.178835,-0.965912,-0.212647,-0.246670
CO-LM,0.002273,0.017157,0.048611,-0.9644,-0.108050,-0.183685,-0.878503,-0.220138,-0.272971
SW-CO-LM,-0.026190,0.011029,0.031250,-0.9582,-0.105197,-0.178835,-0.962477,-0.220504,-0.266442


In [66]:
x[1]

,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero
Lyrics,0.083611,0.022543,0.117222,0.9633,0.098012,0.141572,-0.371796,-0.108447,-0.108447
SW,0.165278,0.028526,0.185417,0.8748,0.035312,0.057381,-0.335312,-0.173610,-0.245097
CO,0.083611,0.022543,0.117222,0.9813,0.125204,0.180850,-0.436700,-0.110671,-0.110671
LM,0.083611,0.022543,0.117222,0.9638,0.098542,0.142339,0.338803,-0.087847,-0.087847
SW-CO,0.165278,0.028526,0.185417,0.9244,0.062504,0.101569,-0.356591,-0.173156,-0.244456
SW-LM,0.165278,0.028526,0.185417,0.8777,0.035842,0.058244,0.336744,-0.155735,-0.219861
CO-LM,0.083611,0.022543,0.117222,0.9816,0.125735,0.181617,-0.294294,-0.089940,-0.089940
SW-CO-LM,0.165278,0.028526,0.185417,0.9265,0.063035,0.102431,0.323240,-0.153256,-0.216362


In [67]:
x[2]

,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero
Lyrics,0.008642,-0.001098,-0.006746,-0.9990,-0.409270,-0.533291,-0.777161,48.027406,107.138060
SW,0.237500,0.019186,0.412500,-0.9945,-0.174158,-0.468050,-0.740457,63.611840,155.495608
CO,0.008642,-0.001098,-0.006746,-0.9990,-0.409270,-0.533291,0.000000,42.190210,92.818462
LM,0.008642,-0.001098,-0.006746,-0.9990,-0.409053,-0.533009,-0.735975,48.037926,107.161526
SW-CO,0.237500,0.019186,0.412500,-0.9945,-0.174158,-0.468050,-0.576116,29.294917,87.884751
SW-LM,0.237500,0.019186,0.412500,-0.9945,-0.173942,-0.467469,-0.797336,63.588578,174.868589
CO-LM,0.008642,-0.001098,-0.006746,-0.9990,-0.409053,-0.533009,0.000000,42.199611,92.839145
SW-CO-LM,0.237500,0.019186,0.412500,-0.9939,-0.167898,-0.451225,-0.761865,29.184158,100.059970


In [68]:
x[3]

,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero
Lyrics,0.131043,0.043190,0.120933,-0.8957,-0.022621,-0.059381,0.476612,0.010187,0.017403
SW,0.136255,0.036312,0.127092,-0.2023,-0.019250,-0.062192,0.873502,0.055158,0.087092
CO,0.131043,0.043190,0.120933,-0.8957,-0.022621,-0.059381,0.487262,-0.011209,-0.017676
LM,0.176857,0.059857,0.193384,-0.8957,-0.022621,-0.059381,0.371561,0.027284,0.041432
SW-CO,0.136255,0.036312,0.127092,-0.2023,-0.019250,-0.062192,0.855904,0.084435,0.126653
SW-LM,0.190284,0.052979,0.222511,-0.2023,-0.019250,-0.062192,0.892811,0.073226,0.109838
CO-LM,0.176857,0.059857,0.193384,-0.8957,-0.022621,-0.059381,0.384594,0.013463,0.019714
SW-CO-LM,0.190284,0.052979,0.222511,-0.6369,-0.031207,-0.100823,0.895829,0.124650,0.169978


In [70]:
x[4]

,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero
Lyrics,0.010779,0.015184,0.024884,0.9222,0.019734,0.029108,-0.462724,25.794275,30.953129
SW,0.022917,0.023093,0.041288,-0.9740,0.021612,0.032695,-0.651018,16.786605,19.664308
CO,0.010779,0.015184,0.024884,0.9339,0.022315,0.032915,-0.358686,25.796120,30.955344
LM,-0.017444,0.038806,0.067341,0.8243,0.009876,0.014568,-0.572606,25.734910,30.210546
SW-CO,0.022917,0.023093,0.041288,-0.9822,0.024193,0.036600,-0.656212,16.787042,19.664820
SW-LM,-0.010227,0.043326,0.082460,-0.9799,0.011754,0.017782,-0.708590,16.777378,19.653500
CO-LM,-0.017444,0.038806,0.067341,0.8243,0.012458,0.018375,-0.475647,25.736755,30.212712
SW-CO-LM,0.004792,0.057733,0.094618,-0.9874,0.013617,0.020600,-0.713785,16.777447,19.653581


Kinda done with this part. Looks okay for now. Now we got to see if we can get an mp3 version of the song and see if the song sounds sad or happy. 

In [ ]:
len("")

0